In [1]:
include("/home/krzysztof/Dokumenty/badania_iitis/tensors_symetric/tensor calculations/calc&codes/sym_tensor/src/Boxtensors.jl")  
include("Test.jl")
using Test

In [2]:
include("/home/krzysztof/Dokumenty/badania_iitis/tensors_symetric/tensor calculations/calc&codes/sym_tensor/src/Boxtensors.jl")
using Boxtensors
importall Boxtensors

In [3]:
i = 1000000

s = 3

for size in [36, 39]
    data = clcopulagen(i, size);
    @time cumulants(3, data, s);
    @time cumulants(4, data, s);
    @time naivecumulant(data, 4);
end

136.956198 seconds (643.61 k allocations: 410.534 GB, 27.74% gc time)
2094.036819 seconds (19.91 M allocations: 6.152 TB, 30.64% gc time)
105512.417406 seconds (218.36 M allocations: 158.878 TB, 54.02% gc time)
163.044710 seconds (736.35 k allocations: 511.432 GB, 27.67% gc time)
2717.331006 seconds (26.34 M allocations: 8.168 TB, 31.00% gc time)
147308.412574 seconds (300.75 M allocations: 218.832 TB, 53.72% gc time)


In [2]:
data = clcopulagen(1000000, 15);

In [5]:
o = 4

@time cumulants(o-1, data, 3);
@time cumulants(o, data, 3);
@time naivecumulant(data, o);

 12.659137 seconds (83.06 k allocations: 41.689 GB, 23.37% gc time)
104.536850 seconds (1.13 M allocations: 343.700 GB, 25.10% gc time)
2976.083995 seconds (6.71 M allocations: 4.789 TB, 49.89% gc time)


In [5]:
for o in collect(6:7)
    @time c = cumulants(o, data, div(size(data, 2), 3));
end
#cn = naivecumulant(data, 5);


 71.303060 seconds (45.45 M allocations: 211.876 GB, 17.23% gc time)
476.877464 seconds (749.81 M allocations: 1006.402 GB, 12.36% gc time)


In [ ]:
 71.781549 seconds (45.45 M allocations: 211.876 GB, 14.92% gc time)
475

#1000000 15 3 
#115.607918 seconds (1.14 M allocations: 343.700 GB, 32.13% gc time)
#3237.854221 seconds (6.58 M allocations: 4.789 TB, 54.23% gc time)
#1000000 30 3 
#1128.944750 seconds (10.66 M allocations: 3.257 TB, 33.08% gc time)
#51451.271712 seconds (105.31 M allocations: 76.620 TB, 54.45% gc time)
#33
#1582.785217 seconds (14.75 M allocations: 4.534 TB, 34.03% gc time)
#78787.640539 seconds (154.18 M allocations: 112.179 TB, 56.60% gc time)
#36
#2146.416321 seconds (19.94 M allocations: 6.152 TB, 34.07% gc time)
#111770.945808 seconds (218.35 M allocations: 158.878 TB, 56.68% gc time)

In [4]:
@time c = snaivecumulant(data, 8)

LoadError: LoadError: UndefVarError: snaivecumulant not defined
while loading In[4], in expression starting on line 155

In [10]:
@time c2, c3 = cumulants(3, data, 5);

  0.068689 seconds (79.27 k allocations: 45.919 MB, 10.93% gc time)


In [9]:
@time c2, c3, c4, c5 = cumulants(5, data, 2);


  0.370966 seconds (342.51 k allocations: 18.139 MB, 1.89% gc time)


In [10]:
cn4 = naivecumulant(data, 4)
@time cn5 = naivecumulant(data, 5);

  0.097818 seconds (330.81 k allocations: 44.802 MB, 8.87% gc time)


In [74]:
@time  cf = get_diff(data, 5);

347.649006 seconds (3.71 M allocations: 1.734 GB, 0.08% gc time)


In [6]:
#138.701652 seconds
#1627.469713 seconds (6.28 G allocations: 595.513 GB, 4.33% gc time)
# 543.527439 seconds (1.89 G allocations: 870.856 GB, 10.93% gc time)
#test = convert(Array, c10) - cf[10]

#test = c["c8"] - convert(Array, c8)
test = convert(Array, c[4]) - cn
maximum(abs(test))


5.329070518200751e-15

In [13]:
#data = clcopulagen(50000, 48);
#272.183187 seconds (61.29 M allocations: 968.278 GB, 25.18% gc time)
# 7585.766758 seconds (716.64 M allocations: 25.126 TB, 20.33% gc time)
# div = 28

In [ ]:
@time bs+bs1;
@time bs+2;
@time st + st1;
@time st + 2;
@time convert(Array, bs);
@time vec(bs);
@time vec(st);
@time bs.*bs1;
@time bs+bs1;
@time bs./bs1;
@time st.*st1;
@time st+st1;
@time st./st1;
@time modemult(bs, c1', 4)
@time Tensors.modemult(st,c1',4);
@time bcssclass(bs, c1')

In [ ]:

if nprocs() == 1
    addprocs()
end
@everywhere begin
    include("/home/krzysztof/Dokumenty/badania_iitis/tensors_symetric/tensor calculations/calc&codes/sym_tensor/src/Boxtensors.jl")
    using Boxtensors
end
importall Boxtensors

In [3]:
x = randn(10, 10);
Base.covm(x, 0; corrected = false)

10x10 Array{Float64,2}:
  1.23845     0.611371   -0.0518148  …  -0.167113    0.116201   0.876525 
  0.611371    1.31396     0.071718      -0.16593    -0.210259   0.314712 
 -0.0518148   0.071718    0.806635       0.448023   -0.355023  -0.34978  
 -0.0571788  -0.130006   -0.334718      -0.298709   -0.19531   -0.15415  
 -0.11585     0.598699   -0.626223      -0.976551    0.496991   0.0645199
 -0.0592483   0.0636601  -0.2557     …   0.180694    0.211843   0.0857358
 -0.276256    0.302982    0.408813       0.958388   -1.23326   -0.412608 
 -0.167113   -0.16593     0.448023       2.17862    -1.18547    0.0491065
  0.116201   -0.210259   -0.355023      -1.18547     1.58335    0.146063 
  0.876525    0.314712   -0.34978        0.0491065   0.146063   0.939336 

In [17]:
methods(Base.covm)


# 4 methods for generic function "covm":
covm(x::AbstractArray{T,1}, xmean) at statistics.jl:277
covm(x::AbstractArray{T,2}, xmean) at statistics.jl:280
covm(x::AbstractArray{T,1}, xmean, y::AbstractArray{T,1}, ymean) at statistics.jl:283
covm(x::Union{AbstractArray{T,1},AbstractArray{T,2}}, xmean, y::Union{AbstractArray{T,1},AbstractArray{T,2}}, ymean) at statistics.jl:286

In [3]:
methods(ForwardDiff.jacobian)

# 5 methods for generic function "jacobian":
jacobian(result::ForwardDiff.ForwardDiffResult{T}) at /home/krzysztof/.julia/v0.4/ForwardDiff/src/api/ForwardDiffResult.jl:84
jacobian(f) at /home/krzysztof/.julia/v0.4/ForwardDiff/src/api/jacobian.jl:49
jacobian{T}(f, x::Array{T,1}) at /home/krzysztof/.julia/v0.4/ForwardDiff/src/api/jacobian.jl:27
jacobian{T,A}(f, x::Array{T,1}, ::Type{A}) at /home/krzysztof/.julia/v0.4/ForwardDiff/src/api/jacobian.jl:27
jacobian{A}(f, ::Type{A}) at /home/krzysztof/.julia/v0.4/ForwardDiff/src/api/jacobian.jl:49

In [1]:
2+2

4

In [5]:
collect(partitions(6,3))

3-element Array{Any,1}:
 [4,1,1]
 [3,2,1]
 [2,2,2]

In [6]:
splitind(n::Vector{Int}, pe::Vector{Vector{Int}}) = map(p->n[p], pe)

function productseg{T <: AbstractFloat}(s::Int, N::Int, part::Vector{Vector{Int}}, c::Array{T}...)
  ret = zeros(T, fill(s, N)...)
  for i = 1:(s^N)
    ind = ind2sub((fill(s, N)...), i)
    pe = splitind(collect(ind), part)
    @inbounds ret[ind...] = mapreduce(i -> c[i][pe[i]...], *, 1:size(part, 1))
  end
  ret
end

sortpart(ls::Vector{Vector{Int}}) = ls[sortperm(map(length, ls))]
si(ls::Vector{Vector{Int}}) = map(length, ls)

function partitionsind(part::Vector{Int})
  ret = Vector{Vector{Int}}[]
  N = sum(part)
  for p in partitions(1:N, length(part))
    p = sortpart(p)
    if (mapreduce(i -> (length(p[i]) == part[i]), *, 1:length(part)))
      push!(ret, p)
    end
  end
  N, ret
end

function findpart(n::Int)
  ret = Vector{Int}[]
  for k = 2:floor(Int, n/2), p in partitions(n, k)
    (1 in p)? (): push!(ret, sort(p))
  end
  ret
end

function part(n::Int, sigma::Int)
    perm = Vector{Vector{Int}}[]
    corder = Vector{Int}[]
    for p in partitions(1:n, sigma)
        s = map(length, p)
        (1 in s)? (): (push!(perm, p), push!(corder, s))
    end
    perm, corder, size(corder, 1)
end
    

part (generic function with 1 method)

In [8]:
a, b, c = part(6,3)
c

15

In [17]:
partitionsind([2,4])

(6,[[[5,6],[1,2,3,4]],[[4,6],[1,2,3,5]],[[4,5],[1,2,3,6]],[[3,6],[1,2,4,5]],[[3,5],[1,2,4,6]],[[3,4],[1,2,5,6]],[[1,2],[3,4,5,6]],[[2,6],[1,3,4,5]],[[2,5],[1,3,4,6]],[[2,4],[1,3,5,6]],[[1,3],[2,4,5,6]],[[2,3],[1,4,5,6]],[[1,4],[2,3,5,6]],[[1,5],[2,3,4,6]],[[1,6],[2,3,4,5]]])

In [18]:
partitionsind([3,7])

(10,[[[8,9,10],[1,2,3,4,5,6,7]],[[7,9,10],[1,2,3,4,5,6,8]],[[7,8,10],[1,2,3,4,5,6,9]],[[7,8,9],[1,2,3,4,5,6,10]],[[6,9,10],[1,2,3,4,5,7,8]],[[6,8,10],[1,2,3,4,5,7,9]],[[6,8,9],[1,2,3,4,5,7,10]],[[6,7,10],[1,2,3,4,5,8,9]],[[6,7,9],[1,2,3,4,5,8,10]],[[6,7,8],[1,2,3,4,5,9,10]]  …  [[1,6,7],[2,3,4,5,8,9,10]],[[1,6,8],[2,3,4,5,7,9,10]],[[1,6,9],[2,3,4,5,7,8,10]],[[1,6,10],[2,3,4,5,7,8,9]],[[1,7,8],[2,3,4,5,6,9,10]],[[1,7,9],[2,3,4,5,6,8,10]],[[1,7,10],[2,3,4,5,6,8,9]],[[1,8,9],[2,3,4,5,6,7,10]],[[1,8,10],[2,3,4,5,6,7,9]],[[1,9,10],[2,3,4,5,6,7,8]]])

In [11]:
splitind([1,2,3,4,5,6], part(6,2)[1])

LoadError: LoadError: MethodError: `splitind` has no method matching splitind(::Array{Int64,1}, ::Array{Array{Array{Int64,1},1},1})
Closest candidates are:
  splitind(::Array{Int64,1}, !Matched::Array{Array{Int64,1},1})
while loading In[11], in expression starting on line 1

In [2]:
a, b = part(6,2)

LoadError: LoadError: UndefVarError: part not defined
while loading In[2], in expression starting on line 1

In [6]:
10 % 3 == 0

false

In [11]:
ceil(Int, 5,2)

LoadError: LoadError: MethodError: `ceil` has no method matching ceil(::Type{Int64}, ::Int64, ::Int64)
Closest candidates are:
  ceil{T<:Integer}(::Type{T<:Integer}, ::Integer)
  ceil{R}(!Matched::AbstractArray{R,1}, ::Integer, ::Integer)
  ceil{R}(!Matched::AbstractArray{R,2}, ::Integer, ::Integer)
  ...
while loading In[11], in expression starting on line 1